# Bishkek Real Estate Price Prediction v2

**Goal:** Predict apartment price per square meter ($/m²) in Bishkek, Kyrgyzstan

**Key Improvements over v1:**
- Temporal train/test split (no future leakage)
- Proper target encoding with K-Fold CV
- Outlier detection with IQR + domain rules
- Feature selection (VIF + importance)
- Ensemble model (XGB + LGBM + CatBoost)
- Prediction intervals (quantile regression)
- Comprehensive residual analysis

In [ ]:
# Install dependencies (for Kaggle)
!pip install -q optuna shap catboost lightgbm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import hashlib

warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, GroupKFold, cross_val_predict
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna

RANDOM_STATE = 42
CURRENT_YEAR = datetime.now().year

print(f"Current year: {CURRENT_YEAR}")
print("Libraries loaded successfully!")

## 1. Load & Clean Data

In [ ]:
import sqlite3
import os

# Load from SQLite database
if os.path.exists('/kaggle/input/bishkek-real-estate-2025/bishkek.db'):
    # Kaggle path
    db_path = '/kaggle/input/bishkek-real-estate-2025/bishkek.db'
elif os.path.exists('../data/databases/bishkek.db'):
    # Local path
    db_path = '../data/databases/bishkek.db'
else:
    raise FileNotFoundError("Database not found!")

conn = sqlite3.connect(db_path)

df_raw = pd.read_sql('''
    SELECT 
        a.*,
        rc.name as jk_name,
        rc.class as jk_class,
        rc.status as jk_status,
        rc.developer_name
    FROM apartments a
    LEFT JOIN residential_complexes rc ON a.residential_complex_id = rc.id
    WHERE a.price_usd IS NOT NULL 
      AND a.area IS NOT NULL
      AND a.price_per_m2 > 0
''', conn)
conn.close()

print(f"Database: {db_path}")
print(f"Raw dataset: {len(df_raw)} rows")
print(f"Columns: {len(df_raw.columns)}")

In [ ]:
# ===================
# 1.1 Remove Duplicates
# ===================
# Same apartment can be listed multiple times with different prices
# Keep the most recent listing

df_raw['parsed_at'] = pd.to_datetime(df_raw['parsed_at'])

# Create building signature for duplicate detection
df_raw['building_signature'] = (
    df_raw['address'].fillna('').str.lower() + '_' +
    df_raw['floor'].fillna(0).astype(str) + '_' +
    df_raw['area'].fillna(0).astype(str) + '_' +
    df_raw['rooms'].fillna(0).astype(str)
)

# Sort by date and keep last (most recent)
df_raw = df_raw.sort_values('parsed_at')
duplicates_before = len(df_raw)
df_raw = df_raw.drop_duplicates(subset=['building_signature'], keep='last')
duplicates_removed = duplicates_before - len(df_raw)

print(f"Duplicates removed: {duplicates_removed}")
print(f"Dataset after dedup: {len(df_raw)} rows")

In [ ]:
# ===================
# 1.2 Outlier Detection (IQR + Domain Rules)
# ===================

target = 'price_per_m2'

def detect_outliers_iqr(series: pd.Series, k: float = 1.5) -> pd.Series:
    """Detect outliers using IQR method"""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - k * IQR
    upper = Q3 + k * IQR
    return (series >= lower) & (series <= upper)

# IQR-based outliers for price
price_mask = detect_outliers_iqr(df_raw[target], k=2.0)

# Domain-based rules
domain_mask = (
    (df_raw[target] >= 300) &      # Min $300/m² (very cheap even for Bishkek)
    (df_raw[target] <= 5000) &     # Max $5000/m² (luxury segment)
    (df_raw['area'] >= 15) &       # Min 15 m² (studio)
    (df_raw['area'] <= 500) &      # Max 500 m² (mansion)
    (df_raw['rooms'].fillna(1) <= 10) &  # Max 10 rooms
    (df_raw['floor'].fillna(1) <= 50)    # Max 50 floors
)

# Combine masks
valid_mask = price_mask & domain_mask
outliers_removed = (~valid_mask).sum()

print(f"Outliers detected: {outliers_removed}")
print(f"  - IQR price outliers: {(~price_mask).sum()}")
print(f"  - Domain rule violations: {(~domain_mask).sum()}")

df = df_raw[valid_mask].copy()
print(f"\nFinal dataset: {len(df)} rows")

In [ ]:
# ===================
# 1.3 Data Quality Report
# ===================

print("=" * 60)
print("DATA QUALITY REPORT")
print("=" * 60)

print(f"\nTarget variable ({target}):")
print(f"  Mean:   ${df[target].mean():,.0f}/m²")
print(f"  Median: ${df[target].median():,.0f}/m²")
print(f"  Std:    ${df[target].std():,.0f}/m²")
print(f"  Range:  ${df[target].min():,.0f} - ${df[target].max():,.0f}/m²")

print(f"\nMissing values (key columns):")
key_cols = ['rooms', 'area', 'floor', 'total_floors', 'year_built', 
            'latitude', 'longitude', 'district', 'jk_name', 'condition']
for col in key_cols:
    if col in df.columns:
        missing = df[col].isna().sum()
        pct = missing / len(df) * 100
        print(f"  {col:20s}: {missing:5d} ({pct:5.1f}%)")

print(f"\nResidential complex coverage:")
print(f"  With JK: {df['jk_name'].notna().sum()} ({df['jk_name'].notna().mean()*100:.1f}%)")
print(f"  Unique JK: {df['jk_name'].nunique()}")

print(f"\nTemporal range:")
print(f"  From: {df['parsed_at'].min()}")
print(f"  To:   {df['parsed_at'].max()}")

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(df[target], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(df[target].median(), color='red', linestyle='--', label=f'Median: ${df[target].median():,.0f}')
axes[0].set_xlabel('Price per m² (USD)')
axes[0].set_title('Price Distribution')
axes[0].legend()

axes[1].hist(np.log1p(df[target]), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1].set_xlabel('Log(Price per m²)')
axes[1].set_title('Log-transformed (more normal)')

# Box plot by district
top_districts = df['district'].value_counts().head(8).index
df_plot = df[df['district'].isin(top_districts)]
df_plot.boxplot(column=target, by='district', ax=axes[2], rot=45)
axes[2].set_title('Price by District')
axes[2].set_xlabel('')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 2. Temporal Train/Test Split

**Critical:** We split by time to avoid future leakage. The model should only see past data during training.

In [ ]:
# ===================
# TEMPORAL SPLIT (not random!)
# ===================

# Sort by date
df = df.sort_values('parsed_at').reset_index(drop=True)

# Use last 20% by time as test set
split_idx = int(len(df) * 0.8)
split_date = df.iloc[split_idx]['parsed_at']

train_df = df.iloc[:split_idx].copy()
test_df = df.iloc[split_idx:].copy()

print(f"Temporal Split:")
print(f"  Train: {len(train_df)} samples ({df['parsed_at'].min().date()} to {split_date.date()})")
print(f"  Test:  {len(test_df)} samples ({split_date.date()} to {df['parsed_at'].max().date()})")
print(f"\nThis ensures no future data leakage!")

## 3. Feature Engineering

**Important:** All statistics (median, encodings) are computed on TRAIN set only!

In [ ]:
# ===================
# POI (Points of Interest) - Bishkek coordinates (verified by user)
# ===================

POI = {
    # Shopping malls
    'dordoi_plaza': (42.8750, 74.6128),
    'bishkek_park': (42.8741, 74.5888),
    'tsum': (42.8746, 74.6031),
    'vefa_center': (42.8668, 74.5931),
    'asia_mall': (42.8489, 74.5672),
    'karavan': (42.8562, 74.5686),
    
    # Key landmarks
    'ala_too_square': (42.8746, 74.6030),
    'philharmonic': (42.8749, 74.6108),
    'white_house': (42.8760, 74.6097),
    'victory_square': (42.8722, 74.5875),
    
    # Universities
    'knu': (42.8778, 74.6027),
    'auca': (42.8634, 74.6167),
    'krsu': (42.8750, 74.5861),
    
    # Transport
    'west_bus_station': (42.8628, 74.5294),
    'east_bus_station': (42.8605, 74.6550),
    'railway_station': (42.8588, 74.6339),
    
    # Markets
    'osh_bazaar': (42.8722, 74.5761),
    'dordoi_bazaar': (42.9453, 74.6494),
    'ortosay_bazaar': (42.8478, 74.5542),
    
    # City center
    'center': (42.8746, 74.5888),
}

PARKS = {
    'dubovy_park': [(42.8749, 74.5875), (42.8780, 74.5875), (42.8780, 74.5930), (42.8749, 74.5930)],
    'park_panfilova': [(42.8740, 74.6000), (42.8760, 74.6000), (42.8760, 74.6050), (42.8740, 74.6050)],
    'park_ataturk': [(42.8690, 74.5950), (42.8720, 74.5950), (42.8720, 74.6000), (42.8690, 74.6000)],
    'botanical_garden': [(42.8560, 74.5560), (42.8620, 74.5560), (42.8620, 74.5660), (42.8560, 74.5660)],
}

ALA_ARCHA_RIVER = [
    (42.7800, 74.5700), (42.8100, 74.5650), (42.8400, 74.5600),
    (42.8600, 74.5580), (42.8800, 74.5560), (42.9000, 74.5550),
]

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate distance between two points in km"""
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def distance_to_polyline(lat, lon, polyline):
    """Minimum distance to polyline"""
    distances = [haversine_distance(lat, lon, p[0], p[1]) for p in polyline]
    return min(distances)

print(f"POI: {len(POI)}, Parks: {len(PARKS)}")

In [ ]:
class FeatureEngineer:
    """
    Feature engineering with proper train/test separation.
    All statistics are computed on train set only.
    """
    
    def __init__(self):
        self.fitted = False
        self.medians = {}
        self.target_encodings = {}
        
    def fit(self, df: pd.DataFrame, target_col: str):
        """Compute all statistics from training data"""
        # Store medians for imputation
        self.medians = {
            'floor': df['floor'].median(),
            'total_floors': df['total_floors'].median(),
            'year_built': df['year_built'].median(),
            'rooms': df['rooms'].median(),
            'kitchen_ratio': (df['kitchen_area'] / df['area']).median(),
            'ceiling_height': self._parse_ceiling_series(df['ceiling_height']).median(),
            'latitude': df['latitude'].median(),
            'longitude': df['longitude'].median(),
        }
        
        # Target encoding will be done separately with CV
        self.global_target_mean = df[target_col].mean()
        self.fitted = True
        return self
    
    def _parse_ceiling_series(self, series: pd.Series) -> pd.Series:
        """Parse ceiling height values"""
        def parse_one(val):
            if pd.isna(val): return np.nan
            val = str(val).replace('м', '').replace(',', '.').strip()
            try: return float(val)
            except: return np.nan
        return series.apply(parse_one)
    
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        """Apply feature engineering using fitted statistics"""
        if not self.fitted:
            raise ValueError("Must call fit() first!")
        
        df = df.copy()
        
        # === Impute missing values with TRAIN medians ===
        df['floor'] = df['floor'].fillna(self.medians['floor'])
        df['total_floors'] = df['total_floors'].fillna(self.medians['total_floors'])
        df['year_built'] = df['year_built'].fillna(self.medians['year_built'])
        df['rooms'] = df['rooms'].fillna(self.medians['rooms'])
        df['latitude'] = df['latitude'].fillna(self.medians['latitude'])
        df['longitude'] = df['longitude'].fillna(self.medians['longitude'])
        
        # === Floor features ===
        df['floor_ratio'] = df['floor'] / df['total_floors'].replace(0, 1)
        df['is_first_floor'] = (df['floor'] == 1).astype(int)
        df['is_last_floor'] = (df['floor'] == df['total_floors']).astype(int)
        
        # === Building features ===
        df['building_age'] = CURRENT_YEAR - df['year_built']
        df['is_new_building'] = (df['year_built'] >= CURRENT_YEAR - 7).astype(int)
        df['is_soviet'] = (df['year_built'] < 1991).astype(int)
        df['is_highrise'] = (df['total_floors'] >= 9).astype(int)
        
        # === Area features ===
        df['area_per_room'] = df['area'] / df['rooms'].replace(0, 1)
        df['is_large'] = (df['area'] >= 100).astype(int)
        df['is_studio'] = (df['rooms'] <= 1).astype(int)
        
        # Kitchen
        df['kitchen_area_num'] = pd.to_numeric(df['kitchen_area'], errors='coerce')
        df['kitchen_ratio'] = (df['kitchen_area_num'] / df['area']).clip(0, 0.5)
        df['kitchen_missing'] = df['kitchen_ratio'].isna().astype(int)
        df['kitchen_ratio'] = df['kitchen_ratio'].fillna(self.medians['kitchen_ratio'])
        
        # Ceiling
        df['ceiling_height_m'] = self._parse_ceiling_series(df['ceiling_height'])
        df['ceiling_missing'] = df['ceiling_height_m'].isna().astype(int)
        df['ceiling_height_m'] = df['ceiling_height_m'].fillna(self.medians['ceiling_height'])
        df['high_ceiling'] = (df['ceiling_height_m'] >= 3.0).astype(int)
        
        # === Condition ===
        condition_map = {'евроремонт': 4, 'хороший': 3, 'средний': 2, 
                        'черновая отделка': 1, 'требует ремонта': 0}
        df['condition_score'] = df['condition'].map(condition_map).fillna(2)
        df['condition_missing'] = (~df['condition'].isin(condition_map.keys())).astype(int)
        
        # === Categorical binary features ===
        df['has_separate_bathroom'] = df['bathroom'].str.contains('раздельн', na=False).astype(int)
        df['has_balcony'] = df['balcony'].notna().astype(int)
        df['has_parking'] = df['parking'].notna().astype(int)
        df['has_furniture'] = df['furniture'].notna().astype(int)
        df['is_parquet'] = df['floor_type'].str.contains('паркет', na=False).astype(int)
        
        # Security score
        df['security_score'] = (
            df['security'].str.contains('охран', na=False).astype(int) * 2 +
            df['security'].str.contains('видео', na=False).astype(int) +
            df['security'].str.contains('домофон', na=False).astype(int)
        )
        
        # House type
        df['is_monolith'] = (df['house_type'] == 'монолитный').astype(int)
        df['is_brick'] = (df['house_type'] == 'кирпичный').astype(int)
        df['is_panel'] = (df['house_type'] == 'панельный').astype(int)
        
        # === JK features ===
        df['has_jk'] = df['jk_name'].notna().astype(int)
        jk_class_map = {'эконом': 1, 'комфорт': 2, 'бизнес': 3, 'премиум': 4, 'элит': 4}
        df['jk_class_score'] = df['jk_class'].map(jk_class_map).fillna(0)
        df['jk_completed'] = (df['jk_status'] == 'completed').astype(int)
        
        # === Geo features ===
        lats = df['latitude'].values
        lons = df['longitude'].values
        
        # POI distances
        for poi_name, (poi_lat, poi_lon) in POI.items():
            df[f'dist_{poi_name}'] = haversine_distance(lats, lons, poi_lat, poi_lon)
        
        # Aggregated distances
        mall_cols = [f'dist_{p}' for p in ['dordoi_plaza', 'bishkek_park', 'tsum', 'vefa_center', 'asia_mall', 'karavan']]
        df['dist_nearest_mall'] = df[mall_cols].min(axis=1)
        
        transport_cols = [f'dist_{p}' for p in ['west_bus_station', 'east_bus_station', 'railway_station']]
        df['dist_nearest_transport'] = df[transport_cols].min(axis=1)
        
        bazaar_cols = [f'dist_{p}' for p in ['osh_bazaar', 'dordoi_bazaar', 'ortosay_bazaar']]
        df['dist_nearest_bazaar'] = df[bazaar_cols].min(axis=1)
        
        # Parks
        for park_name, polygon in PARKS.items():
            centroid = (np.mean([p[0] for p in polygon]), np.mean([p[1] for p in polygon]))
            df[f'dist_{park_name}'] = haversine_distance(lats, lons, centroid[0], centroid[1])
        
        park_cols = [f'dist_{p}' for p in PARKS.keys()]
        df['dist_nearest_park'] = df[park_cols].min(axis=1)
        
        # River distance
        df['dist_river'] = df.apply(
            lambda r: distance_to_polyline(r['latitude'], r['longitude'], ALA_ARCHA_RIVER), axis=1
        )
        
        return df
    
    def fit_transform(self, df: pd.DataFrame, target_col: str) -> pd.DataFrame:
        return self.fit(df, target_col).transform(df)

In [ ]:
# ===================
# Target Encoding with K-Fold CV (no leakage)
# ===================

class TargetEncoderCV:
    """
    Target encoding with K-Fold cross-validation to prevent leakage.
    For training data: use out-of-fold predictions
    For test data: use full training set statistics
    """
    
    def __init__(self, cols: List[str], min_samples: int = 30, n_folds: int = 5):
        self.cols = cols
        self.min_samples = min_samples
        self.n_folds = n_folds
        self.encodings = {}
        self.global_mean = None
        
    def fit_transform(self, df: pd.DataFrame, target_col: str) -> pd.DataFrame:
        """Fit on training data and transform with CV (no leakage)"""
        df = df.copy()
        self.global_mean = df[target_col].mean()
        
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=RANDOM_STATE)
        
        for col in self.cols:
            new_col = f'{col}_target_enc'
            df[new_col] = np.nan
            
            # Fill with out-of-fold encoding
            for train_idx, val_idx in kf.split(df):
                train_fold = df.iloc[train_idx]
                val_fold = df.iloc[val_idx]
                
                # Compute encoding from train fold
                encoding = self._compute_encoding(train_fold, col, target_col)
                
                # Apply to validation fold
                df.iloc[val_idx, df.columns.get_loc(new_col)] = (
                    val_fold[col].map(encoding).fillna(self.global_mean)
                )
            
            # Store full encoding for test set
            self.encodings[col] = self._compute_encoding(df, col, target_col)
        
        return df
    
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        """Transform test data using full training encodings"""
        df = df.copy()
        
        for col in self.cols:
            new_col = f'{col}_target_enc'
            df[new_col] = df[col].map(self.encodings[col]).fillna(self.global_mean)
        
        return df
    
    def _compute_encoding(self, df: pd.DataFrame, col: str, target_col: str) -> Dict:
        """Compute smoothed target encoding"""
        agg = df.groupby(col)[target_col].agg(['mean', 'count'])
        
        # Smoothing formula: more samples = more trust in category mean
        smoothing = agg['count'] / (agg['count'] + self.min_samples)
        encoding = smoothing * agg['mean'] + (1 - smoothing) * self.global_mean
        
        return encoding.to_dict()

print("Target Encoder with CV defined")

In [ ]:
# ===================
# Apply Feature Engineering
# ===================

# 1. Basic features (fit on train only)
feature_engineer = FeatureEngineer()
train_df = feature_engineer.fit_transform(train_df, target)
test_df = feature_engineer.transform(test_df)

print(f"Basic features created: {train_df.shape[1]} columns")

# 2. Target encoding (with CV on train, direct on test)
target_encoder = TargetEncoderCV(
    cols=['district', 'jk_name'], 
    min_samples=30,  # Higher threshold for better smoothing
    n_folds=5
)
train_df = target_encoder.fit_transform(train_df, target)
test_df = target_encoder.transform(test_df)

print(f"Target encoding applied")
print(f"Final train shape: {train_df.shape}")
print(f"Final test shape: {test_df.shape}")

## 4. Feature Selection

In [ ]:
# ===================
# Define Feature Groups
# ===================

# Core features (no multicollinearity)
FEATURE_GROUPS = {
    'core': ['rooms', 'area', 'floor_ratio', 'building_age'],
    
    'building': ['is_new_building', 'is_soviet', 'is_highrise', 'total_floors'],
    
    'apartment': [
        'area_per_room', 'is_large', 'is_studio',
        'kitchen_ratio', 'kitchen_missing',
        'ceiling_height_m', 'ceiling_missing', 'high_ceiling',
        'condition_score', 'condition_missing',
    ],
    
    'amenities': [
        'has_separate_bathroom', 'has_balcony', 'has_parking',
        'has_furniture', 'is_parquet', 'security_score',
    ],
    
    'house_type': ['is_monolith', 'is_brick', 'is_panel'],
    
    'jk': ['has_jk', 'jk_class_score', 'jk_completed'],
    
    'target_encoding': ['district_target_enc', 'jk_name_target_enc'],
    
    'location': [
        'dist_center', 'dist_nearest_mall', 'dist_nearest_transport',
        'dist_nearest_bazaar', 'dist_nearest_park', 'dist_river',
        'latitude', 'longitude',
    ],
}

# Flatten to get all features
ALL_FEATURES = [f for group in FEATURE_GROUPS.values() for f in group]

# Check availability
available_features = [f for f in ALL_FEATURES if f in train_df.columns]
missing_features = [f for f in ALL_FEATURES if f not in train_df.columns]

print(f"Features defined: {len(ALL_FEATURES)}")
print(f"Features available: {len(available_features)}")
if missing_features:
    print(f"Missing: {missing_features}")

In [ ]:
# ===================
# Check Multicollinearity (VIF)
# ===================

from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df: pd.DataFrame, features: List[str]) -> pd.DataFrame:
    """Calculate Variance Inflation Factor for features"""
    X = df[features].fillna(0).values
    vif_data = []
    
    for i, feature in enumerate(features):
        try:
            vif = variance_inflation_factor(X, i)
            vif_data.append({'feature': feature, 'VIF': vif})
        except:
            vif_data.append({'feature': feature, 'VIF': np.inf})
    
    return pd.DataFrame(vif_data).sort_values('VIF', ascending=False)

# Check VIF for numeric features
numeric_features = [f for f in available_features if train_df[f].dtype in ['float64', 'int64']]
vif_df = calculate_vif(train_df, numeric_features[:20])  # Check top 20

print("Top features by VIF (>5 indicates multicollinearity):")
print(vif_df.head(10).to_string(index=False))

In [ ]:
# ===================
# Remove High VIF Features
# ===================

# Remove features with VIF > 10 (keeping the most important variant)
high_vif_features = vif_df[vif_df['VIF'] > 10]['feature'].tolist()

# Manual selection: keep the most informative from correlated pairs
features_to_remove = []
# year_built and building_age are correlated - keep building_age
if 'year_built' in available_features:
    features_to_remove.append('year_built')
# floor and floor_ratio - keep floor_ratio
if 'floor' in available_features and 'floor_ratio' in available_features:
    features_to_remove.append('floor')

# Final feature list
final_features = [f for f in available_features if f not in features_to_remove]

print(f"Features removed due to multicollinearity: {features_to_remove}")
print(f"Final feature count: {len(final_features)}")

In [ ]:
# ===================
# Prepare Final Data
# ===================

X_train = train_df[final_features].fillna(0).values
X_test = test_df[final_features].fillna(0).values
y_train = train_df[target].values
y_test = test_df[target].values

# Create groups for CV (by building location)
# Hash of (lat, lon) rounded to create spatial groups
def create_spatial_groups(df: pd.DataFrame, precision: int = 3) -> np.ndarray:
    """Create groups based on spatial proximity"""
    lat_round = df['latitude'].round(precision).astype(str)
    lon_round = df['longitude'].round(precision).astype(str)
    group_str = lat_round + '_' + lon_round
    
    le = LabelEncoder()
    return le.fit_transform(group_str)

train_groups = create_spatial_groups(train_df)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"Spatial groups for CV: {len(np.unique(train_groups))}")

## 5. Model Training & Ensemble

In [ ]:
# ===================
# Evaluation Metrics
# ===================

def evaluate_predictions(y_true: np.ndarray, y_pred: np.ndarray, name: str = "") -> Dict:
    """Comprehensive evaluation metrics"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    # Median Absolute Error (robust to outliers)
    medae = np.median(np.abs(y_true - y_pred))
    
    # Median Absolute Percentage Error (better than MAPE)
    mape = np.median(np.abs((y_true - y_pred) / y_true)) * 100
    
    # Percentage within 10% error
    within_10pct = np.mean(np.abs((y_true - y_pred) / y_true) <= 0.10) * 100
    
    return {
        'name': name,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MedAE': medae,
        'MedAPE%': mape,
        'Within10%': within_10pct,
    }

def print_metrics(metrics: Dict):
    print(f"  MAE:       ${metrics['MAE']:,.0f}/m²")
    print(f"  RMSE:      ${metrics['RMSE']:,.0f}/m²")
    print(f"  R²:        {metrics['R²']:.4f}")
    print(f"  MedAE:     ${metrics['MedAE']:,.0f}/m²")
    print(f"  MedAPE:    {metrics['MedAPE%']:.1f}%")
    print(f"  Within 10%: {metrics['Within10%']:.1f}%")

In [ ]:
# ===================
# Train Base Models
# ===================

print("Training base models...\n")

base_models = {
    'XGBoost': XGBRegressor(
        n_estimators=300, max_depth=8, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        random_state=RANDOM_STATE, n_jobs=-1
    ),
    'LightGBM': LGBMRegressor(
        n_estimators=300, max_depth=8, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        random_state=RANDOM_STATE, verbose=-1, n_jobs=-1
    ),
    'CatBoost': CatBoostRegressor(
        n_estimators=300, max_depth=8, learning_rate=0.05,
        random_state=RANDOM_STATE, verbose=0
    ),
}

results = []
trained_models = {}
oof_predictions = {}  # Out-of-fold predictions for stacking

for name, model in base_models.items():
    print(f"{name}:")
    
    # Train
    model.fit(X_train, y_train)
    trained_models[name] = model
    
    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # OOF predictions using GroupKFold
    gkf = GroupKFold(n_splits=5)
    oof = cross_val_predict(model, X_train, y_train, cv=gkf, groups=train_groups, n_jobs=-1)
    oof_predictions[name] = oof
    
    # Evaluate
    metrics = evaluate_predictions(y_test, y_pred_test, name)
    results.append(metrics)
    print_metrics(metrics)
    print()

results_df = pd.DataFrame(results).sort_values('MAE')
print("\n" + "="*60)
print("BASE MODELS COMPARISON")
print("="*60)
print(results_df.to_string(index=False))

In [ ]:
# ===================
# Stacking Ensemble
# ===================

print("\nTraining Stacking Ensemble...")

# Prepare stacking features (OOF predictions)
stack_train = np.column_stack([oof_predictions[name] for name in base_models.keys()])
stack_test = np.column_stack([trained_models[name].predict(X_test) for name in base_models.keys()])

# Meta-learner (Ridge regression - simple and effective)
meta_model = Ridge(alpha=1.0)
meta_model.fit(stack_train, y_train)

# Ensemble prediction
y_pred_ensemble = meta_model.predict(stack_test)

# Evaluate ensemble
ensemble_metrics = evaluate_predictions(y_test, y_pred_ensemble, 'Stacking Ensemble')
print("\nStacking Ensemble Results:")
print_metrics(ensemble_metrics)

# Print weights
print(f"\nMeta-learner weights:")
for name, weight in zip(base_models.keys(), meta_model.coef_):
    print(f"  {name}: {weight:.3f}")

In [ ]:
# ===================
# Simple Average Ensemble (often works better)
# ===================

y_pred_avg = np.mean([
    trained_models[name].predict(X_test) for name in base_models.keys()
], axis=0)

avg_metrics = evaluate_predictions(y_test, y_pred_avg, 'Average Ensemble')
print("\nSimple Average Ensemble Results:")
print_metrics(avg_metrics)

# Choose best ensemble
if avg_metrics['MAE'] < ensemble_metrics['MAE']:
    print("\n>>> Using Simple Average (better performance)")
    final_predictions = y_pred_avg
    final_method = 'average'
else:
    print("\n>>> Using Stacking Ensemble (better performance)")
    final_predictions = y_pred_ensemble
    final_method = 'stacking'

## 6. Prediction Intervals (Quantile Regression)

In [ ]:
# ===================
# Train Quantile Models for Prediction Intervals
# ===================

print("Training quantile models for prediction intervals...")

# LightGBM supports quantile regression natively
quantile_models = {}

for q, name in [(0.10, 'q10'), (0.50, 'q50'), (0.90, 'q90')]:
    model = LGBMRegressor(
        objective='quantile',
        alpha=q,
        n_estimators=200,
        max_depth=8,
        learning_rate=0.05,
        random_state=RANDOM_STATE,
        verbose=-1
    )
    model.fit(X_train, y_train)
    quantile_models[name] = model
    print(f"  {name} trained")

# Get prediction intervals
pred_q10 = quantile_models['q10'].predict(X_test)
pred_q50 = quantile_models['q50'].predict(X_test)
pred_q90 = quantile_models['q90'].predict(X_test)

# Check coverage (should be ~80% for 10-90 interval)
coverage = np.mean((y_test >= pred_q10) & (y_test <= pred_q90)) * 100
avg_interval_width = np.mean(pred_q90 - pred_q10)

print(f"\nPrediction Intervals (80% CI):")
print(f"  Coverage: {coverage:.1f}% (target: 80%)")
print(f"  Average interval width: ${avg_interval_width:,.0f}/m²")

## 7. Residual Analysis

In [ ]:
# ===================
# Comprehensive Residual Analysis
# ===================

residuals = y_test - final_predictions
std_residuals = (residuals - residuals.mean()) / residuals.std()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Residuals vs Predicted (check heteroscedasticity)
axes[0, 0].scatter(final_predictions, residuals, alpha=0.3, s=10)
axes[0, 0].axhline(0, color='red', linestyle='--')
axes[0, 0].set_xlabel('Predicted')
axes[0, 0].set_ylabel('Residual')
axes[0, 0].set_title('Residuals vs Predicted\n(check for patterns)')

# 2. Residuals distribution
axes[0, 1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(0, color='red', linestyle='--')
axes[0, 1].axvline(residuals.mean(), color='green', linestyle='--', label=f'Mean: ${residuals.mean():,.0f}')
axes[0, 1].set_xlabel('Residual')
axes[0, 1].set_title('Residuals Distribution')
axes[0, 1].legend()

# 3. QQ Plot (check normality)
from scipy import stats
stats.probplot(std_residuals, dist="norm", plot=axes[0, 2])
axes[0, 2].set_title('Q-Q Plot\n(should be linear if normal)')

# 4. Actual vs Predicted
axes[1, 0].scatter(y_test, final_predictions, alpha=0.3, s=10)
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual')
axes[1, 0].set_ylabel('Predicted')
axes[1, 0].set_title('Actual vs Predicted')

# 5. Residuals vs Actual (check for bias at different price levels)
axes[1, 1].scatter(y_test, residuals, alpha=0.3, s=10)
axes[1, 1].axhline(0, color='red', linestyle='--')
axes[1, 1].set_xlabel('Actual Price')
axes[1, 1].set_ylabel('Residual')
axes[1, 1].set_title('Residuals vs Actual\n(check for bias)')

# 6. Prediction intervals
sample_idx = np.random.choice(len(y_test), min(100, len(y_test)), replace=False)
sample_idx = np.sort(sample_idx)

axes[1, 2].fill_between(range(len(sample_idx)), 
                        pred_q10[sample_idx], pred_q90[sample_idx], 
                        alpha=0.3, label='80% CI')
axes[1, 2].scatter(range(len(sample_idx)), y_test[sample_idx], s=20, c='red', label='Actual')
axes[1, 2].plot(range(len(sample_idx)), pred_q50[sample_idx], 'b-', label='Median pred')
axes[1, 2].set_xlabel('Sample')
axes[1, 2].set_ylabel('Price/m²')
axes[1, 2].set_title('Prediction Intervals (sample)')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

In [ ]:
# ===================
# Error Analysis by Segment
# ===================

test_analysis = test_df.copy()
test_analysis['predicted'] = final_predictions
test_analysis['residual'] = residuals
test_analysis['abs_error'] = np.abs(residuals)
test_analysis['pct_error'] = test_analysis['abs_error'] / test_analysis[target] * 100

print("=" * 60)
print("ERROR ANALYSIS BY SEGMENT")
print("=" * 60)

# By district
print("\nBy District:")
district_error = test_analysis.groupby('district').agg({
    'abs_error': ['mean', 'median'],
    'pct_error': 'median',
    target: 'count'
}).round(1)
district_error.columns = ['MAE', 'MedAE', 'MedAPE%', 'Count']
district_error = district_error.sort_values('MedAE')
print(district_error.head(10))

# By JK presence
print("\nBy Residential Complex:")
jk_error = test_analysis.groupby('has_jk').agg({
    'abs_error': ['mean', 'median'],
    'pct_error': 'median',
    target: 'count'
}).round(1)
jk_error.columns = ['MAE', 'MedAE', 'MedAPE%', 'Count']
jk_error.index = ['No JK', 'Has JK']
print(jk_error)

# By price segment
print("\nBy Price Segment:")
test_analysis['price_segment'] = pd.cut(
    test_analysis[target], 
    bins=[0, 1000, 1500, 2000, 3000, 10000],
    labels=['<$1k', '$1-1.5k', '$1.5-2k', '$2-3k', '>$3k']
)
segment_error = test_analysis.groupby('price_segment').agg({
    'abs_error': ['mean', 'median'],
    'pct_error': 'median',
    target: 'count'
}).round(1)
segment_error.columns = ['MAE', 'MedAE', 'MedAPE%', 'Count']
print(segment_error)

## 8. Feature Importance

In [ ]:
# ===================
# Permutation Importance (more reliable than built-in)
# ===================

from sklearn.inspection import permutation_importance

print("Computing permutation importance...")

# Use XGBoost for importance analysis
perm_importance = permutation_importance(
    trained_models['XGBoost'], X_test, y_test,
    n_repeats=10, random_state=RANDOM_STATE, n_jobs=-1
)

importance_df = pd.DataFrame({
    'feature': final_features,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std,
}).sort_values('importance_mean', ascending=False)

# Normalize to percentage
importance_df['importance_pct'] = importance_df['importance_mean'] / importance_df['importance_mean'].sum() * 100

print("\nTop 15 Features (Permutation Importance):")
print(importance_df.head(15).to_string(index=False))

In [ ]:
# Visualize importance with error bars
fig, ax = plt.subplots(figsize=(10, 8))

top_n = 15
top_features = importance_df.head(top_n)

y_pos = np.arange(top_n)
ax.barh(y_pos, top_features['importance_mean'], 
        xerr=top_features['importance_std'], 
        align='center', alpha=0.8)
ax.set_yticks(y_pos)
ax.set_yticklabels(top_features['feature'])
ax.invert_yaxis()
ax.set_xlabel('Permutation Importance (MAE increase)')
ax.set_title('Top 15 Feature Importances with Std Dev')

plt.tight_layout()
plt.show()

## 9. Save Model

In [ ]:
import joblib
import json

# Save all components
model_artifacts = {
    'base_models': trained_models,
    'meta_model': meta_model if final_method == 'stacking' else None,
    'ensemble_method': final_method,
    'quantile_models': quantile_models,
    'feature_engineer': feature_engineer,
    'target_encoder': target_encoder,
    'features': final_features,
}

joblib.dump(model_artifacts, 'bishkek_model_v2.joblib')
print("Model artifacts saved to: bishkek_model_v2.joblib")

# Save config
final_metrics = evaluate_predictions(y_test, final_predictions, 'Final')

config = {
    'version': 2,
    'features': final_features,
    'ensemble_method': final_method,
    'metrics': {
        'mae': float(final_metrics['MAE']),
        'rmse': float(final_metrics['RMSE']),
        'r2': float(final_metrics['R²']),
        'medae': float(final_metrics['MedAE']),
        'medape': float(final_metrics['MedAPE%']),
        'within_10pct': float(final_metrics['Within10%']),
        'ci_coverage': float(coverage),
    },
    'data_info': {
        'train_size': len(train_df),
        'test_size': len(test_df),
        'train_date_range': f"{train_df['parsed_at'].min().date()} to {train_df['parsed_at'].max().date()}",
        'test_date_range': f"{test_df['parsed_at'].min().date()} to {test_df['parsed_at'].max().date()}",
    }
}

with open('bishkek_model_v2_config.json', 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print("Config saved to: bishkek_model_v2_config.json")

## 10. Summary

In [ ]:
print("="*70)
print("BISHKEK REAL ESTATE PRICE PREDICTION v2 - SUMMARY")
print("="*70)

print(f"\n[DATA]")
print(f"  Total samples: {len(df):,}")
print(f"  Train: {len(train_df):,} | Test: {len(test_df):,}")
print(f"  Split method: Temporal (last 20% by date)")
print(f"  JK coverage: {df['jk_name'].notna().mean()*100:.1f}%")

print(f"\n[FEATURES]")
print(f"  Total features: {len(final_features)}")
print(f"  Target encoding: CV-based (no leakage)")
print(f"  Top 3: {', '.join(importance_df['feature'].head(3).tolist())}")

print(f"\n[MODEL]")
print(f"  Ensemble: {final_method.title()} (XGBoost + LightGBM + CatBoost)")
print(f"  Prediction intervals: 80% CI (Quantile Regression)")

print(f"\n[PERFORMANCE]")
print(f"  MAE:        ${final_metrics['MAE']:,.0f}/m²")
print(f"  MedAE:      ${final_metrics['MedAE']:,.0f}/m²")
print(f"  R²:         {final_metrics['R²']:.3f}")
print(f"  MedAPE:     {final_metrics['MedAPE%']:.1f}%")
print(f"  Within 10%: {final_metrics['Within10%']:.1f}%")
print(f"  CI Coverage: {coverage:.1f}% (target: 80%)")

print(f"\n[INTERPRETATION]")
median_price = df[target].median()
print(f"  Median price: ${median_price:,.0f}/m²")
print(f"  Typical error: ${final_metrics['MedAE']:,.0f}/m² ({final_metrics['MedAE']/median_price*100:.1f}%)")
print(f"  For 60m² apartment (~${median_price*60:,.0f}):")
print(f"    Expected error: ~${final_metrics['MedAE']*60:,.0f}")

print(f"\n[IMPROVEMENTS OVER v1]")
print(f"  + Temporal split (no future leakage)")
print(f"  + Target encoding with CV (no train leakage)")
print(f"  + Proper outlier detection (IQR + domain rules)")
print(f"  + Duplicate removal")
print(f"  + Ensemble model (3 algorithms)")
print(f"  + Prediction intervals (quantile regression)")
print(f"  + Comprehensive residual analysis")
print(f"  + Permutation importance (more reliable)")

print("\n" + "="*70)